## BTC-USD Price Returns Direction Prediction
1. Approach 1: Based on Forecasted Value by our Existing Model
   1. Data Preparation
   2. Classification
   3. Evaluation
      1. Classification Error
      2. Better than 0.5 (Coin Flip)?


2. [KIV] Approach 2: Train a classifier using Market and Uncertainty Indices data

In [1]:
# NB config
%load_ext autoreload
%autoreload 2

# Load Libraries
import os
import importlib
import warnings
os.chdir("../../")
print(os.getcwd())
import numpy as np
import pandas as pd
from typing import List, Dict
from pathlib import Path
from sklearn.metrics import classification_report

dm_test = importlib.import_module("Diebold-Mariano-Test.dm_test")

warnings.filterwarnings("ignore")

/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index


### 1. Approach 1

#### Data Preparation
* Generate Price Returns for h = 1 to 12 (Y-Actual)
* Get Forecasts for h = 1 to 12 (Y-Pred)
* Create 12 dataframes for each pair of (Y-Actual, Y-Pred)

In [2]:
def gen_log_price_returns(
    series: pd.DataFrame, h: int, var_col: str = "Price", drop_cols: List[str] = ['Volume']
) -> pd.DataFrame:
    new_col_name = "price_return"
    series[new_col_name] = np.log1p(series[[var_col]].pct_change(h))
    drop_cols.append(var_col)
    return series.drop(columns=drop_cols).rename(columns={'Date': 'time'})

In [21]:
# Load Data
# Data Dir
data_dir = Path("forecasting/data/modelling")

# BTC-USD data
btc_usd_fp = data_dir / "btc_usd_weekly.csv"
btc_usd_df = pd.read_csv(btc_usd_fp)

# Generate Log Price Returns for multi-horizons
y_actual_dict = {}

for i in range(0, 12):
    y_actual_dict[i] = (
        gen_log_price_returns(btc_usd_df, i + 1)
    )

In [22]:
# Generate Directions

y_actual_clf_dict = {}

for i, df in y_actual_dict.items():
    df['direction'] = df['price_return'].apply(lambda x: 'positive' if x >= 0 else 'negative')
    y_actual_clf_dict[i] = df

In [23]:
# Get Predictions for h = 1 to 12
forecast_dir = Path("forecasting/data/forecasts/random_forest")

def get_forecast_fps(dir: Path, keyword: str = 'rf_model') -> List[Path]:
    return (
        list(filter(lambda x: keyword in str(x), list(dir.glob("*.csv"))))
    )

# Model A
model_A_dir = forecast_dir / "model_A"
model_A_forecast_dict = {
    k: pd.read_csv(fp) for k, fp
    in enumerate(get_forecast_fps(model_A_dir))
}

# Model B
model_B_dir = forecast_dir / "model_B"
model_B_forecast_dict = {
    k: pd.read_csv(fp) for k, fp
    in enumerate(get_forecast_fps(model_B_dir))
}

# Model C
model_C_dir = forecast_dir / "model_C"
model_C_forecast_dict = {
    k: pd.read_csv(fp) for k, fp
    in enumerate(get_forecast_fps(model_C_dir))
}

# Model D
model_D_dir = forecast_dir / "model_D"
model_D_forecast_dict = {
    k: pd.read_csv(fp) for k, fp
    in enumerate(get_forecast_fps(model_D_dir))
}

# Model E
model_E_dir = forecast_dir / "model_E"
model_E_forecast_dict = {
    k: pd.read_csv(fp) for k, fp
    in enumerate(get_forecast_fps(model_E_dir))
}

# Model F
model_F_dir = forecast_dir / "model_F"
model_F_forecast_dict = {
    k: pd.read_csv(fp) for k, fp
    in enumerate(get_forecast_fps(model_F_dir))
}

### Classification

In [25]:
# Process Forecast Dict helper

def classify_multi_h(forecast_dict: Dict[int, pd.DataFrame], col_name: str = 'price_return', threshold: float = 0.0):
    results = {}
    for k, df in forecast_dict.items():
        df['direction'] = df[col_name].apply(lambda x: 'positive' if x >= threshold else 'negative')
        results[k] = df
    return results

In [29]:
# Compute classification results

def get_clf_res(truth_dict, pred_dict, clf_col: str = 'direction'):
    assert truth_dict.keys() == pred_dict.keys()
    reports = {}
    for i in list(truth_dict.keys()):
        combined_df = pred_dict[i].merge(truth_dict[i], on='time', suffixes=('_pred', '_truth')).dropna()
        reports[i + 1] = pd.DataFrame(
            classification_report(combined_df[clf_col + '_pred'], combined_df[clf_col + '_truth'], output_dict=True)
        )
    return reports

In [33]:
# Model A
model_A_clf = classify_multi_h(
    model_A_forecast_dict
)

model_A_results = get_clf_res(y_actual_clf_dict, model_A_clf)

# Inspect Results for h = 1, 4 and 12
print(model_A_results[1])
print(model_A_results[4])
print(model_A_results[12])

            negative   positive  accuracy   macro avg  weighted avg
precision   0.280702   0.661972  0.492188    0.471337      0.542825
recall      0.400000   0.534091  0.492188    0.467045      0.492188
f1-score    0.329897   0.591195  0.492188    0.460546      0.509539
support    40.000000  88.000000  0.492188  128.000000    128.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.418182   0.666667  0.559055    0.542424      0.574708
recall      0.489362   0.600000  0.559055    0.544681      0.559055
f1-score    0.450980   0.631579  0.559055    0.541280      0.564743
support    47.000000  80.000000  0.559055  127.000000    127.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.681818   0.786517   0.75188    0.734168      0.747944
recall      0.612245   0.833333   0.75188    0.722789      0.751880
f1-score    0.645161   0.809249   0.75188    0.727205      0.748795
support    49.000000  84.000000   0.75188  133.0

In [34]:
# Model B
model_B_clf = classify_multi_h(
    model_B_forecast_dict
)

model_B_results = get_clf_res(y_actual_clf_dict, model_B_clf)

# Inspect Results for h = 1, 4 and 12
print(model_B_results[1])
print(model_B_results[4])
print(model_B_results[12])

            negative   positive  accuracy   macro avg  weighted avg
precision   0.321429   0.591549  0.472441    0.456489      0.491583
recall      0.382979   0.525000  0.472441    0.453989      0.472441
f1-score    0.349515   0.556291  0.472441    0.452903      0.479768
support    47.000000  80.000000  0.472441  127.000000    127.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.385965   0.794521  0.615385    0.590243      0.678239
recall      0.594595   0.623656  0.615385    0.609125      0.615385
f1-score    0.468085   0.698795  0.615385    0.583440      0.633132
support    37.000000  93.000000  0.615385  130.000000    130.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.863636   0.952381  0.921875    0.908009      0.923262
recall      0.904762   0.930233  0.921875    0.917497      0.921875
f1-score    0.883721   0.941176  0.921875    0.912449      0.922324
support    42.000000  86.000000  0.921875  128.0

In [35]:
# Model C
model_C_clf = classify_multi_h(
    model_C_forecast_dict
)

model_C_results = get_clf_res(y_actual_clf_dict, model_C_clf)

# Inspect Results for h = 1, 4 and 12
print(model_C_results[1])
print(model_C_results[4])
print(model_C_results[12])

            negative   positive  accuracy   macro avg  weighted avg
precision   0.321429   0.608696      0.48    0.465062      0.505280
recall      0.400000   0.525000      0.48    0.462500      0.480000
f1-score    0.356436   0.563758      0.48    0.460097      0.489122
support    45.000000  80.000000      0.48  125.000000    125.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.388889   0.694444  0.563492    0.541667      0.590168
recall      0.488372   0.602410  0.563492    0.545391      0.563492
f1-score    0.432990   0.645161  0.563492    0.539075      0.572754
support    43.000000  83.000000  0.563492  126.000000    126.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.363636   0.673913  0.573529    0.518775      0.568967
recall      0.347826   0.688889  0.573529    0.518357      0.573529
f1-score    0.355556   0.681319  0.573529    0.518437      0.571134
support    46.000000  90.000000  0.573529  136.0

In [36]:
# Model D
model_D_clf = classify_multi_h(
    model_D_forecast_dict
)

model_D_results = get_clf_res(y_actual_clf_dict, model_D_clf)

# Inspect Results for h = 1, 4 and 12
print(model_D_results[1])
print(model_D_results[4])
print(model_D_results[12])

            negative   positive  accuracy   macro avg  weighted avg
precision   0.372881   0.630137  0.515152    0.501509      0.534641
recall      0.448980   0.554217  0.515152    0.501598      0.515152
f1-score    0.407407   0.589744  0.515152    0.498575      0.522058
support    49.000000  83.000000  0.515152  132.000000    132.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.385965   0.794521  0.615385    0.590243      0.678239
recall      0.594595   0.623656  0.615385    0.609125      0.615385
f1-score    0.468085   0.698795  0.615385    0.583440      0.633132
support    37.000000  93.000000  0.615385  130.000000    130.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.818182   0.894118  0.868217    0.856150      0.867628
recall      0.800000   0.904762  0.868217    0.852381      0.868217
f1-score    0.808989   0.899408  0.868217    0.854199      0.867867
support    45.000000  84.000000  0.868217  129.0

In [37]:
# Model E
model_E_clf = classify_multi_h(
    model_E_forecast_dict
)

model_E_results = get_clf_res(y_actual_clf_dict, model_E_clf)

# Inspect Results for h = 1, 4 and 12
print(model_E_results[1])
print(model_E_results[4])
print(model_E_results[12])

            negative   positive  accuracy   macro avg  weighted avg
precision   0.315789   0.625000  0.488372    0.470395      0.517136
recall      0.400000   0.535714  0.488372    0.467857      0.488372
f1-score    0.352941   0.576923  0.488372    0.464932      0.498790
support    45.000000  84.000000  0.488372  129.000000    129.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.370370   0.647887     0.528    0.509129      0.547981
recall      0.444444   0.575000     0.528    0.509722      0.528000
f1-score    0.404040   0.609272     0.528    0.506656      0.535388
support    45.000000  80.000000     0.528  125.000000    125.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.545455   0.788889  0.708955    0.667172      0.710772
recall      0.558140   0.780220  0.708955    0.669180      0.708955
f1-score    0.551724   0.784530  0.708955    0.668127      0.709824
support    43.000000  91.000000  0.708955  134.0

In [38]:
# Model F
model_F_clf = classify_multi_h(
    model_F_forecast_dict
)

model_F_results = get_clf_res(y_actual_clf_dict, model_F_clf)

# Inspect Results for h = 1, 4 and 12
print(model_F_results[1])
print(model_F_results[4])
print(model_F_results[12])

            negative    positive  accuracy   macro avg  weighted avg
precision   0.216667    0.723684       0.5    0.470175      0.596930
recall      0.382353    0.539216       0.5    0.460784      0.500000
f1-score    0.276596    0.617978       0.5    0.447287      0.532632
support    34.000000  102.000000       0.5  136.000000    136.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.578947   0.810811  0.709924    0.694879      0.727623
recall      0.702128   0.714286  0.709924    0.708207      0.709924
f1-score    0.634615   0.759494  0.709924    0.697055      0.714690
support    47.000000  84.000000  0.709924  131.000000    131.000000
            negative   positive  accuracy   macro avg  weighted avg
precision   0.909091   0.888889     0.896    0.898990      0.896808
recall      0.816327   0.947368     0.896    0.881847      0.896000
f1-score    0.860215   0.917197     0.896    0.888706      0.894860
support    49.000000  76.000000     0.896  